In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import scipy.signal as signal

In [ ]:
# Function to apply a bandpass filter to the audio signal
def apply_bandpass_filter(y, sr, low_freq, high_freq):
    nyquist = 0.5 * sr
    low = low_freq / nyquist
    high = high_freq / nyquist
    b, a = signal.butter(1, [low, high], btype='band')
    y_filtered = signal.lfilter(b, a, y)
    return y_filtered

# Function to extract relevant audio features
def extract_audio_features_with_filters(directory):
    features = []
    for foldername in os.listdir(directory):
        folder_path = os.path.join(directory, foldername)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.wav'):
                    file_path = os.path.join(folder_path, filename)
                    # Load the audio file
                    y, sr = librosa.load(file_path, sr=16000)
                    
                    # Apply bandpass filter (e.g., 300 Hz to 3400 Hz for human voice)
                    y_filtered = apply_bandpass_filter(y, sr, low_freq=300, high_freq=3400)
                    
                    # Extract MFCC features (using the filtered signal)
                    mfcc = librosa.feature.mfcc(y=y_filtered, sr=sr, n_mfcc=13)
                    mfcc_mean = np.mean(mfcc, axis=1)  # Mean MFCC across time
                    
                    # Extract Chroma features
                    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
                    chroma_mean = np.mean(chroma, axis=1)
                    
                    # Extract Spectral features
                    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
                    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
                    spectral_flatness = librosa.feature.spectral_flatness(y=y)
                    
                    # Calculate the mean and standard deviation for spectral features
                    spectral_centroid_mean = np.mean(spectral_centroid)
                    spectral_bandwidth_mean = np.mean(spectral_bandwidth)
                    spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
                    spectral_flatness_mean = np.mean(spectral_flatness)
                    
                    # Append extracted features to the list
                    features.append([
                        foldername, 
                        *mfcc_mean, 
                        *chroma_mean, 
                        spectral_centroid_mean, 
                        spectral_bandwidth_mean, 
                        *spectral_contrast_mean, 
                        spectral_flatness_mean
                    ])
    
    # Create a DataFrame for the extracted features
    mfcc_columns = [f'MFCC_{i}' for i in range(1, 14)]
    chroma_columns = [f'Chroma_{i}' for i in range(1, 13)]
    spectral_columns = ['Spectral_Centroid', 'Spectral_Bandwidth', 'Spectral_Contrast1', 
                        'Spectral_Contrast2', 'Spectral_Contrast3', 'Spectral_Contrast4', 
                        'Spectral_Contrast5', 'Spectral_Contrast6', 'Spectral_Contrast7', 
                        'Spectral_Flatness']
    
    columns = ['Class'] + mfcc_columns + chroma_columns + spectral_columns
    return pd.DataFrame(features, columns=columns)

# Assuming your dataset is stored in 'dataset_path'
dataset_path = 'data'
audio_features_df = extract_audio_features_with_filters(dataset_path)

# Displaying the first few rows of the extracted features
print(audio_features_df.head())

In [ ]:
# Encode class labels as integers (manual label encoding)
classes = audio_features_df['Class'].unique()
class_to_index = {label: index for index, label in enumerate(classes)}
audio_features_df['Class'] = audio_features_df['Class'].map(class_to_index)

# Split the dataset into 80% train and 20% test
def train_test_split(data, test_size=0.2):
    np.random.seed(42)  # For reproducibility
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_size)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# Splitting the dataset
train_data, test_data = train_test_split(audio_features_df)

# Separate features and labels
X_train = train_data.drop(columns=['Class']).values
y_train = train_data['Class'].values
X_test = test_data.drop(columns=['Class']).values
y_test = test_data['Class'].values

In [ ]:
class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.mean = np.zeros((len(self.classes), n_features), dtype=np.float64)
        self.var = np.zeros((len(self.classes), n_features), dtype=np.float64)
        self.priors = np.zeros(len(self.classes), dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

# Train Naive Bayes
nb = NaiveBayes()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

In [ ]:
class DecisionTree:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            return self._most_common_label(y)

        feat_idxs = np.random.choice(n_features, n_features, replace=False)

        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return (best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thresh in thresholds:
                gain = self._information_gain(y, X_column, thresh)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = thresh

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n, n_left, n_right = len(y), len(left_idxs), len(right_idxs)
        e_left, e_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        return parent_entropy - child_entropy

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _predict(self, inputs):
        node = self.tree
        while isinstance(node, tuple):
            if inputs[node[0]] <= node[1]:
                node = node[2]
            else:
                node = node[3]
        return node

# Train Decision Tree
dt = DecisionTree()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [ ]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            idxs = np.random.choice(len(X), len(X), replace=True)
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X[idxs], y[idxs])
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.array([np.bincount(tree_preds[:, i]).argmax() for i in range(X.shape[0])])

# Train Random Forest
rf = RandomForest()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [ ]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        y_ = np.where(y > 0, 1, 0)

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self._activation_function(linear_output)

                update = self.lr * (y_[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def _activation_function(self, x):
        return np.where(x >= 0, 1, 0)

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self._activation_function(linear_output)

# Train Perceptron
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
y_pred_perceptron = perceptron.predict(X_test)

In [ ]:
# Accuracy function
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

# Evaluate models
accuracy_nb = accuracy(y_test, y_pred_nb)
accuracy_dt = accuracy(y_test, y_pred_dt)
accuracy_rf = accuracy(y_test, y_pred_rf)
accuracy_perceptron = accuracy(y_test, y_pred_perceptron)

# Display results
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")
print(f"Decision Tree Accuracy: {accuracy_dt * 100:.2f}%")
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")
print(f"Perceptron Accuracy: {accuracy_perceptron * 100:.2f}%")